**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [6]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape,BatchNormalization, LeakyReLU, Flatten

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [7]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [8]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> In reinforcement learning, there is the exploration / exploitation dilemma. If we only take actions that are optimal according to our current estimate of the action value fonction or value fonction, we won't visit all the state action pairs and we won't be able to improve our estimate of the value function (Statue-action value function). The optimal states actions according to our current estimate of the state-action value function is probably not accurate and we need to keep visiting all state action pairs to assymptotically discover the true optimal state action values.

> However because we want the agent to become good at the task as soon as posssible, we don't want him to act completely randomly forever. This is especially true when we take into consideration the performance of the agent during training (for instance for an agent that is doing a certain task and tries to improve it's performance while being "on production"). Therefore we use what we call an epsilon greedy policy : the agent makes a fraction of the decisions randomly and acts according to his current policy / optimaly with regard to his current estimate of the state action value function the rest of the time. Epsilon usually converges towards zero during training. 

> What's more, once the agent's etimation of the value function start's being somehow good, following an epsilon greedy policy instead of a completely random policy guides the agent's exploration towards paths that are more likely to be "good.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [9]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [10]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=25 # set small when debugging
epochs_test= 5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

>The array board represents the repartition of the cheese and the poison on the map. It equivalently represnts the rewards associated with reaching each category location (+0.5 for cheese, and -1 for poison). After cheese or poison is collected, the value on the corresponding entree of the board array is switched to 0 (no more cheese or poison on this cell).

>The position board represents the limits of the map (-1 for "walls" or "limits of the island"). It also uses a 1 to signify the previous position of the agent.

>Both are used to produce the state object which is a subsampling of those two arrays around the position of the agent.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [11]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__(epsilon=1)
        pass

    def learned_act(self, s):
        return np.random.randint(0,self.n_action,1) 

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [12]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
            
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
          
        while not game_over:
            
            # The agent performs an action
            action = agent.act(state, train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

    

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [13]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/14.0. Average score (-4.5)
Win/lose count 12.5/13.0. Average score (-2.5)
Win/lose count 8.0/12.0. Average score (-3.0)
Win/lose count 9.5/17.0. Average score (-4.125)
Win/lose count 9.5/12.0. Average score (-3.8)
Final score: -3.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [13]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        while len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        n = np.random.randint(0, len(self.memory))
        return self.memory[n]

***
The pipeline we will use for training is given below:

In [14]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [59]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        

    def learned_act(self, s):
        q_s = self.model.predict(s[np.newaxis])[0]
        a = np.argmax(q_s)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            [s, ns, a, r, game_over] = self.memory.random_access()
            input_states[i] = s
            target_q[i] = self.model.predict(s[np.newaxis])
            if game_over_:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount * np.max(self.model.predict(ns[np.newaxis]))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile(Adam(lr=0.0001, "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, layers = 2, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(64))
        model.add(LeakyReLU(alpha = 0.1))
        for _ in range(layers-1):
            model.add(Dense(64))
            model.add(LeakyReLU(alpha = 0.1))
        model.add(Dense(self.n_action))
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model
        

In [56]:
# parameters
size = 13
T=200
temperature= 0.3
epochs_train= 201 # set small when debugging
epochs_test= 5 # set small when debugging

In [57]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/201 | Loss 0.0294 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/201 | Loss 0.0823 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/201 | Loss 0.1524 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/201 | Loss 0.1839 | Win/lose count 0.5/3.0 (-2.5)
Epoch 004/201 | Loss 0.1947 | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/201 | Loss 0.2700 | Win/lose count 1.5/0 (1.5)
Epoch 006/201 | Loss 0.3274 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/201 | Loss 0.3660 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/201 | Loss 0.4030 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/201 | Loss 0.5976 | Win/lose count 5.5/4.0 (1.5)
Epoch 010/201 | Loss 0.8976 | Win/lose count 2.0/2.0 (0.0)
Epoch 011/201 | Loss 1.2279 | Win/lose count 2.0/2.0 (0.0)
Epoch 012/201 | Loss 1.5540 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/201 | Loss 1.5405 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/201 | Loss 1.5491 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/201 | Loss 1.4735 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/201 | Loss 1.8438 | Win/lose count 3.0/5.0 (-

Epoch 138/201 | Loss 1.7505 | Win/lose count 5.0/1.0 (4.0)
Epoch 139/201 | Loss 1.7254 | Win/lose count 11.5/1.0 (10.5)
Epoch 140/201 | Loss 1.7663 | Win/lose count 7.5/2.0 (5.5)
Epoch 141/201 | Loss 1.7554 | Win/lose count 13.5/6.0 (7.5)
Epoch 142/201 | Loss 1.7599 | Win/lose count 9.0/1.0 (8.0)
Epoch 143/201 | Loss 1.7402 | Win/lose count 12.5/0 (12.5)
Epoch 144/201 | Loss 1.7635 | Win/lose count 8.5/1.0 (7.5)
Epoch 145/201 | Loss 1.7565 | Win/lose count 15.5/2.0 (13.5)
Epoch 146/201 | Loss 1.6851 | Win/lose count 11.5/0 (11.5)
Epoch 147/201 | Loss 1.8353 | Win/lose count 8.0/6.0 (2.0)
Epoch 148/201 | Loss 1.7632 | Win/lose count 17.0/7.0 (10.0)
Epoch 149/201 | Loss 1.7852 | Win/lose count 23.0/1.0 (22.0)
Epoch 150/201 | Loss 1.7021 | Win/lose count 8.0/1.0 (7.0)
Epoch 151/201 | Loss 1.6817 | Win/lose count 11.0/5.0 (6.0)
Epoch 152/201 | Loss 1.7850 | Win/lose count 12.0/7.0 (5.0)
Epoch 153/201 | Loss 1.7215 | Win/lose count 16.0/6.0 (10.0)
Epoch 154/201 | Loss 1.6996 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [60]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, 2, input_shape=(5,5,self.n_state)))
        model.add(LeakyReLU(alpha = 0.1))
        model.add(Conv2D(64, 2))
        model.add(Flatten())
        model.add(Dense(self.n_action))
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [63]:
# parameters
size = 13
T=200
temperature= 0.3
epochs_train= 201 # set small when debugging
epochs_test= 20 # set small when debugging

In [141]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size = 10000, batch_size = 64)
train(agent,env,epochs_train ,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/201 | Loss 0.0161 | Win/lose count 3.5/4.0 (-0.5)
Epoch 001/201 | Loss 0.1486 | Win/lose count 6.0/4.0 (2.0)
Epoch 002/201 | Loss 1.9087 | Win/lose count 5.5/2.0 (3.5)
Epoch 003/201 | Loss 1.9631 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/201 | Loss 1.8814 | Win/lose count 6.5/4.0 (2.5)
Epoch 005/201 | Loss 1.8759 | Win/lose count 6.0/2.0 (4.0)
Epoch 006/201 | Loss 1.8098 | Win/lose count 10.0/1.0 (9.0)
Epoch 007/201 | Loss 1.8049 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/201 | Loss 1.8704 | Win/lose count 5.5/2.0 (3.5)
Epoch 009/201 | Loss 1.8587 | Win/lose count 3.5/4.0 (-0.5)
Epoch 010/201 | Loss 1.8156 | Win/lose count 6.0/4.0 (2.0)
Epoch 011/201 | Loss 1.8409 | Win/lose count 6.5/3.0 (3.5)
Epoch 012/201 | Loss 1.7722 | Win/lose count 13.5/2.0 (11.5)
Epoch 013/201 | Loss 1.7956 | Win/lose count 12.5/3.0 (9.5)
Epoch 014/201 | Loss 1.7331 | Win/lose count 8.5/7.0 (1.5)
Epoch 015/201 | Loss 1.7738 | Win/lose count 8.5/1.0 (7.5)
Epoch 016/201 | Loss 1.7422 | Win/lose count 10.5/

Epoch 136/201 | Loss 1.7794 | Win/lose count 18.5/5.0 (13.5)
Epoch 137/201 | Loss 1.7757 | Win/lose count 22.0/6.0 (16.0)
Epoch 138/201 | Loss 1.7820 | Win/lose count 12.0/1.0 (11.0)
Epoch 139/201 | Loss 1.7816 | Win/lose count 24.5/5.0 (19.5)
Epoch 140/201 | Loss 1.7478 | Win/lose count 13.0/2.0 (11.0)
Epoch 141/201 | Loss 1.8116 | Win/lose count 22.5/3.0 (19.5)
Epoch 142/201 | Loss 1.7517 | Win/lose count 21.5/5.0 (16.5)
Epoch 143/201 | Loss 1.7698 | Win/lose count 24.0/7.0 (17.0)
Epoch 144/201 | Loss 1.7445 | Win/lose count 20.0/2.0 (18.0)
Epoch 145/201 | Loss 1.7647 | Win/lose count 20.0/2.0 (18.0)
Epoch 146/201 | Loss 1.7790 | Win/lose count 13.5/3.0 (10.5)
Epoch 147/201 | Loss 1.7697 | Win/lose count 12.5/4.0 (8.5)
Epoch 148/201 | Loss 1.7461 | Win/lose count 20.0/6.0 (14.0)
Epoch 149/201 | Loss 1.7651 | Win/lose count 24.5/5.0 (19.5)
Epoch 150/201 | Loss 1.7871 | Win/lose count 20.0/3.0 (17.0)
Epoch 151/201 | Loss 1.7473 | Win/lose count 19.0/5.0 (14.0)
Epoch 152/201 | Loss 1.75

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [82]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.5/0. Average score (7.5)
Win/lose count 21.0/0. Average score (14.25)
Win/lose count 17.0/0. Average score (15.166666666666666)
Win/lose count 5.0/0. Average score (12.625)
Win/lose count 9.5/0. Average score (12.0)
Win/lose count 16.0/0. Average score (12.666666666666666)
Win/lose count 8.5/0. Average score (12.071428571428571)
Win/lose count 17.5/1.0. Average score (12.625)
Win/lose count 3.5/0. Average score (11.61111111111111)
Win/lose count 2.5/0. Average score (10.7)
Win/lose count 8.5/0. Average score (10.5)
Win/lose count 15.0/0. Average score (10.875)
Win/lose count 19.5/0. Average score (11.538461538461538)
Win/lose count 8.5/0. Average score (11.321428571428571)
Win/lose count 3.5/0. Average score (10.8)
Win/lose count 14.0/0. Average score (11.0)
Win/lose count 6.0/0. Average score (10.705882352941176)
Win/lose count 8.0/0. Average score (10.555555555555555)
Win/lose count 19.5/0. Average score (11.026315789473685)
Win/lose count 11.5/0. Ave

In [65]:
HTML(display_videos('cnn_test10.mp4'))

In [72]:
HTML(display_videos('fc_test10.mp4'))

> Both agents are almost perfect at avoiding poison. However the CNN Agent is much better than the FC agent.

> I observe the following issues :  
   - Lack of exploration : no incentive to go and discover new areas of the map, no memory of what has already been visited further than one step behind. As a consequence the agent eats the cheese arround him and then gets stuck in a loop (moving back and fourh in between two positions or in a circle around four positions...)
   - Because of the random behavior during training and because we do not decrease epsilon towards 0 during training, the agent learns to stay far away from poison (to make sure a random step won't make him touch it). For instance. it can be hesitant to go and get some cheese that is stuck between two poisons.
   - The fully connected agent is quite bad at learning to go towards cheese that is 3 or 4 steps away from him (corners of his vision field). I blame this on the low value of epsilon throughout the entire training. It is often stuck in loops during training and does not explore enough to learnhow to go to cheese that is this far.
   - Cheese acts as a sort of bait. It's an incentive for the agent to move further. Therefore the agent is quite good when temperature is high : there is a lof of cheese and he keeps chasing the next cheese and ends up collecting a lot of it. When temperature is low and cheese is rare, it has no incentive to explore the map, gets stuck and collects very little cheese.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [43]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Update the epsilon for the epsilong greedy policy
            agent.set_epsilon(np.exp(-e/40))
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.visited = np.zeros((grid_size, grid_size))
        
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 1] = 256
        b[self.visited>0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]= 0
        b[:,-2:,:]= 0
        b[:2,:,:]= 0
        b[:,:2,:]= 0

        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        #During training we penalise going back to a visited state
        if train:
            reward = - self.visited[self.x, self.y] * 0.1
        #We tag the new position as visited
        self.visited[self.x, self.y] = 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited[:,:,np.newaxis]),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        #We tag the initial position as visited
        self.visited = np.zeros((self.grid_size, self.grid_size))
        self.visited[self.x, self.y] = 1
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited[:,:,np.newaxis]),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [42]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/201 | Loss 0.8327 | Win/lose count 12.0/24.900000000000045 (-12.900000000000045)
Epoch 001/201 | Loss 1.8130 | Win/lose count 7.5/28.300000000000068 (-20.800000000000068)
Epoch 002/201 | Loss 1.8786 | Win/lose count 10.5/27.500000000000075 (-17.000000000000075)
Epoch 003/201 | Loss 2.0222 | Win/lose count 7.0/25.000000000000085 (-18.000000000000085)
Epoch 004/201 | Loss 1.8799 | Win/lose count 8.5/28.300000000000075 (-19.800000000000075)
Epoch 005/201 | Loss 1.8665 | Win/lose count 8.5/23.300000000000082 (-14.800000000000082)
Epoch 006/201 | Loss 1.7605 | Win/lose count 9.0/25.900000000000034 (-16.900000000000034)
Epoch 007/201 | Loss 1.8404 | Win/lose count 13.0/26.900000000000055 (-13.900000000000055)
Epoch 008/201 | Loss 1.8129 | Win/lose count 14.5/28.500000000000075 (-14.000000000000075)
Epoch 009/201 | Loss 1.7780 | Win/lose count 16.5/21.80000000000001 (-5.300000000000011)
Epoch 010/201 | Loss 1.7781 | Win/lose count 9.0/26.000000000000078 (-17.000000000000078)
Epoch 0

Epoch 093/201 | Loss 1.3500 | Win/lose count 19.5/15.699999999999978 (3.800000000000022)
Epoch 094/201 | Loss 1.2277 | Win/lose count 17.5/12.099999999999975 (5.400000000000025)
Epoch 095/201 | Loss 1.2965 | Win/lose count 23.5/10.89999999999998 (12.60000000000002)
Epoch 096/201 | Loss 1.0106 | Win/lose count 21.5/11.999999999999977 (9.500000000000023)
Epoch 097/201 | Loss 1.1505 | Win/lose count 20.0/12.299999999999972 (7.700000000000028)
Epoch 098/201 | Loss 1.0844 | Win/lose count 20.5/13.499999999999972 (7.000000000000028)
Epoch 099/201 | Loss 0.9973 | Win/lose count 22.5/11.399999999999979 (11.100000000000021)
Epoch 100/201 | Loss 1.2548 | Win/lose count 23.0/14.799999999999974 (8.200000000000026)
Epoch 101/201 | Loss 1.0050 | Win/lose count 24.0/9.99999999999998 (14.00000000000002)
Epoch 102/201 | Loss 1.1300 | Win/lose count 22.5/14.899999999999975 (7.6000000000000245)
Epoch 103/201 | Loss 1.0537 | Win/lose count 23.0/12.799999999999974 (10.200000000000026)
Epoch 104/201 | Loss 

Epoch 185/201 | Loss 1.1382 | Win/lose count 24.5/11.999999999999977 (12.500000000000023)
Epoch 186/201 | Loss 1.0039 | Win/lose count 22.0/11.799999999999981 (10.200000000000019)
Epoch 187/201 | Loss 1.1593 | Win/lose count 23.5/8.499999999999986 (15.000000000000014)
Epoch 188/201 | Loss 1.0115 | Win/lose count 19.5/13.599999999999968 (5.900000000000032)
Epoch 189/201 | Loss 1.1997 | Win/lose count 32.5/9.19999999999999 (23.30000000000001)
Epoch 190/201 | Loss 1.0382 | Win/lose count 23.5/10.599999999999978 (12.900000000000022)
Epoch 191/201 | Loss 1.2531 | Win/lose count 24.0/11.499999999999975 (12.500000000000025)
Epoch 192/201 | Loss 0.9399 | Win/lose count 18.0/13.599999999999971 (4.400000000000029)
Epoch 193/201 | Loss 0.8891 | Win/lose count 19.0/12.99999999999997 (6.00000000000003)
Epoch 194/201 | Loss 1.0017 | Win/lose count 21.5/10.699999999999978 (10.800000000000022)
Epoch 195/201 | Loss 0.7544 | Win/lose count 27.5/11.299999999999976 (16.200000000000024)
Epoch 196/201 | Los

In [48]:
# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_cnn_exp = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
agent_cnn_exp.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')

test(agent_cnn_exp ,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 21.0/2.0. Average score (19.0)
Win/lose count 17.0/2.0. Average score (17.0)
Win/lose count 20.0/0. Average score (18.0)
Win/lose count 12.0/2.0. Average score (16.0)
Win/lose count 16.0/0. Average score (16.0)
Win/lose count 27.5/2.0. Average score (17.583333333333332)
Win/lose count 26.0/1.0. Average score (18.642857142857142)
Win/lose count 16.5/0. Average score (18.375)
Win/lose count 4.5/0. Average score (16.833333333333332)
Win/lose count 23.0/0. Average score (17.45)
Win/lose count 13.0/2.0. Average score (16.863636363636363)
Win/lose count 20.0/1.0. Average score (17.041666666666668)
Win/lose count 21.5/1.0. Average score (17.307692307692307)
Win/lose count 16.0/0. Average score (17.214285714285715)
Win/lose count 23.0/1.0. Average score (17.533333333333335)
Win/lose count 16.0/3.0. Average score (17.25)
Win/lose count 23.0/3.0. Average score (17.41176470588235)
Win/lose count 25.0/1.0. Average score (17.77777777777778)
Win/lose count 22.5/1.0. Average score (17.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***